## Files to Import

In [ ]:
project_name = 'Mason'
# Name of project for file saving and graph title purposes

labeled_seq_csv_file=

umap_reductions_folder=

unique_positive_fv =r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/outfile/mHER_H3_AgPos_unique_fv.csv'

unique_negative_fv = r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/outfile/mHER_H3_AgNeg_unique_fv.csv'

selected_graphs_csv = r'/Users/isaacdaviet/Desktop/graphs/clustering/mason_umap_clustering_analysis_binders.csv'
# csv file containing ranges to test/plot. column format: [distance metric, start neighbors, end neighbors, neighbors step, start dist, end dist, dist step]. INCLUDE THESE EXACT COLUMN NAMES

## Setup

In [ ]:
import sys
sys.path.append('/Users/isaacdaviet/Desktop/thesis/python versions')
# replace with directory containing the .py calculation files below
import clustering_functions as clustering
import pandas as pd

## Add iSeq Identifiers to Labeled Sequences CSV File

In [ ]:
clustering.add_iseq_to_labels_file(labeled_seq_csv_file, unique_positive_fv, unique_negative_fv)

# Optimize clustering


MISSING: STRUCTURE OF CSV FILE

Run each umap selection individually until desired clusters are acheived. When satified, switch 'generate' column to done.

Steps:
- select which clustering algorithm to use and which point type (binders/non binders) to use
- In the 'selected_graphs_csv' file, fill the generate column with 'y' for the graphs you would like to optimize
- Fill in the variables in the csv file for the appropriate algorithm (dbscan_eps + dbscan_min_samples for dbscan and agg_n_clusters and agg_linkage for agglomerative)
- Save changes to csv file and run cell below
- Analyze resulting plotly and adjust parameters until satisified with the clustering
- Use the 'combine_clusters' column to specify which clusters, if any, should be merged. Note: specifically designed for   clusters that are difficult to cluster as a single group and that would be classified as 'low priority' in the QC steps
- When satisfied, switch 'generate' to 'done' and proceed to next graph


In [ ]:
save_folder = r'/Users/isaacdaviet/Desktop/results/clustering/PCA_dbscan_clusters'
#save folder for graphs and cvs files.

algorithm = 'dbscan'
# Which clustering algorithm to use. As it stands now, dbscan and agglomerative are the only options available.


clustering.cluster_extract_and_plot(selected_graphs_csv, labeled_seq_csv_file, umap_reductions_folder, clusters = points_to_optimize, algorithm = algorithm, generate_indicator = 'y', plt_size = [60, 20], pt_size = 30, fontsize=64, show_unclustered_pts='n' ,save_graph = 'n', save_folder = save_folder, project_name=project_name, save_clusters_csv = 'n', save_plotly = 'y')


# Full Clustering

Produces csv, plotly and png graphs of all clusters indicated by 'done' in the generate column. Png images contain original binder/non binder + binder clusters + non binder clusters graphs as a single image.

NOTE: cluster graphs in png images DO NOT display any unclustered points, but the plotly graphs do.

In [ ]:
save_folder = r'/Users/isaacdaviet/Desktop/graphs/clustering/dbscan_clusters'
#save folder for graphs and cvs files.

points_to_optimize = 'both'
# what points to show on graphs. 'both'/'all' shows binders and non binders in same graph, 'binders' show only binders, 'non binders' shows only non binders

algorithm = 'dbscan'

### Final save (csv, html plotly and subplots)
clustering.cluster_extract_and_plot(selected_graphs_csv, labeled_seq_csv_file, umap_reductions_folder,
                                     clusters = points_to_optimize, 
                                     algorithm = algorithm,
                                     generate_indicator = 'done', # Do not change
                                     plt_size = [60, 20],
                                     pt_size = 20,
                                     fontsize=64,
                                     show_unclustered_pts='n', # Recommend not to change as unclustered points make analysis difficult
                                     save_graph = 'n', # Save resulting graph as a .png
                                     save_folder = save_folder, project_name=project_name, save_clusters_csv = 'y',
                                     save_plotly = 'n') # save resulting graph as a plotly

# Manual Cluster Adjustement & Classification

Modifies cluster dataframes to create add 'adjusted_clusters', which updates the clusters to reflect the manual cluster combination specified in the combine_clusters column of the cluster_analysis csv file, and a 'priority' column that classifies each point in a cluster as high/med/low/out_of_bounds priority based on percentages used as input parameters.

In [ ]:
### Manual inspection of a specific data frame's clusters as a percentage of the total number of points for a specific LABEL (ie: (# binders in cluster / total # of binders) * 100)

clusters_df = r'/Users/isaacdaviet/Desktop/graphs/clustering/correlation_clusters/csv_files/UMAP_Mason-correlation-3-25-0.0-1-2_dbscanClusters-0.15-20.csv'


points = 'non binders' # point type to analyze ('all', 'Binder', 'Non Binder')

clusters_df = pd.read_csv(clusters_df)

priorities = clustering.cluster_priority(clusters_df, 
                                         high_threshold = 10, # above = high priority 
                                         low_threshold = 1, # Below = low piority
                                         upper_limit = None, # above = out_of_bounds
                                         print_percentages = points)

In [ ]:
### Automatically applies cluster adjustment and prioritization to all csv_files in a folder. Thresholds/upper limit are set as percentages

high_threshold = 10
low_threshold = 1
upper_limit = 40


clusters_csv_folder = r'/Users/isaacdaviet/Desktop/graphs/clustering/dbscan_clusters'

clustering.adjust_clusters_folder(clusters_csv_folder, selected_graphs_csv, point_type = 'Binder', high_threshold = high_threshold, low_threshold = low_threshold, upper_limit = upper_limit)